In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
import warnings

In [2]:
warnings.filterwarnings("ignore")
rcParams['figure.figsize'] = 12, 8

In [3]:
def show_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)
        
def generate_date_features(df, fldname, drop=True):
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True, errors=errors)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
#     if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre+"Is_weekend"]=df[targ_pre+"Dayofweek"].apply(lambda x:1 if x==5 or x==6 else 0)
#     df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)   
    
def join_df(left, right, left_on, right_on=None, suffix='_y'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, suffixes=("",suffix))

In [4]:
%%time
train = pd.read_csv('data/Data_Train.csv')
test = pd.read_csv('data/Data_Test.csv')
submission = pd.read_csv('data/Sample_Submission.csv')

CPU times: user 302 ms, sys: 61.5 ms, total: 363 ms
Wall time: 362 ms


In [5]:
train.shape, test.shape, submission.shape

((147443, 15), (49148, 14), (49148, 7))

In [6]:
show_all(train.head())

,ID,Date,LicenseNo,FacilityID,FacilityName,Type,Street,City,State,LocationID,Reason,SectionViolations,RiskLevel,Geo_Loc,Inspection_Results
0,31103489027986,26-04-2010,4744,8123,7715,RESTAURANT,15522,id-11235901,id_1890134,81876.0,CANVASS,33.0,High,locid16406,4
1,10088999935915,21-06-2009,2973,12268,11664,GROCERY STORE,3057,id-11235901,id_1890134,81862.0,COMPLAINT,33.0,High,locid878,4
2,40148966010272,01-05-2013,18223,1112,969,RESTAURANT,14988,id-11235901,id_1890134,81883.0,CANVASS,NaN,High,locid3368,6
3,37157708563676,28-09-2015,20825,20007,19115,RESTAURANT,3661,id-11235901,id_1890134,81859.0,CANVASS RE-INSPECTION,31.0,Medium,locid11839,4
4,47478049564374,09-12-2015,2136,16867,10409,RESTAURANT,7876,id-11235901,id_1890134,81886.0,COMPLAINT,30.0,High,locid12264,4


In [7]:
show_all(submission.head())

,FACILITY CHANGED,FAIL,FURTHER INSPECTION REQUIRED,INSPECTION OVERRULED,PASS,PASS(CONDITIONAL),SHUT-DOWN
0,0.1,0.1,0.1,0.1,0.1,0.1,0.1
1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
2,0.1,0.1,0.1,0.1,0.1,0.1,0.1
3,0.1,0.1,0.1,0.1,0.1,0.1,0.1
4,0.1,0.1,0.1,0.1,0.1,0.1,0.1


In [8]:
for df in (train, test):
    df['Date'] = df['Date'].replace('29-02-2011', '28-02-2011')
    df['Date'] = df['Date'].replace('29-02-2015', '28-02-2015')
    df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y', errors='coerce')

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147443 entries, 0 to 147442
Data columns (total 15 columns):
ID                    147443 non-null int64
Date                  147443 non-null datetime64[ns]
LicenseNo             147443 non-null int64
FacilityID            147443 non-null int64
FacilityName          147443 non-null int64
Type                  143958 non-null object
Street                147443 non-null int64
City                  147443 non-null object
State                 147443 non-null object
LocationID            147408 non-null float64
Reason                147443 non-null object
SectionViolations     108375 non-null float64
RiskLevel             147443 non-null object
Geo_Loc               147443 non-null object
Inspection_Results    147443 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(6), object(6)
memory usage: 16.9+ MB


In [10]:
show_all(train.iloc[:,1:].describe())

,LicenseNo,FacilityID,FacilityName,Street,LocationID,SectionViolations,Inspection_Results
count,147443.000000,147443.000000,147443.000000,147443.000000,147408.000000,108375.000000,147443.000000
mean,17726.613003,14020.779583,13377.891009,8684.119653,81873.642156,24.129153,3.680405
std,10865.050018,7794.382869,7408.148606,5255.991577,140.248846,12.657456,1.481441
min,0.000000,0.000000,0.000000,0.000000,31259.000000,1.000000,0.000000
25%,8588.000000,7347.000000,6993.000000,4109.000000,81859.000000,14.000000,4.000000
50%,17056.000000,14256.000000,13710.000000,8541.000000,81870.000000,31.000000,4.000000
75%,26163.000000,20841.000000,19795.000000,13321.000000,81888.000000,33.000000,4.000000
max,37394.000000,27276.000000,26032.000000,17891.000000,82072.000000,70.000000,6.000000


In [11]:
for df in (train, test):
    missing_stats = df.isnull().sum()
    print(missing_stats[missing_stats != 0])

Type                  3485
LocationID              35
SectionViolations    39068
dtype: int64
Type                  1215
LocationID              15
SectionViolations    12976
dtype: int64


In [12]:
test['Inspection_Results'] = -1

In [13]:
full_data = train.append(test).reset_index(drop=True)
full_data.shape

(196591, 15)

## Exploratory Analysis

In [14]:
id_col = ['ID']
date_col = ['Date']
cat_cols = ['LicenseNo', 'FacilityID', 'FacilityName', 'Type', 'Street', 'City', 'State', 'LocationID', 'Reason', 'RiskLevel', 'Geo_Loc']
target_col = ['Inspection_Results']

In [15]:
for col in full_data.columns:
    print(f"{col}: {full_data[col].nunique()}")

ID: 196591
Date: 2521
LicenseNo: 37395
FacilityID: 27277
FacilityName: 26033
Type: 435
Street: 17892
City: 2
State: 2
LocationID: 112
Reason: 17
SectionViolations: 62
RiskLevel: 4
Geo_Loc: 16812
Inspection_Results: 8


In [16]:
for col in (cat_cols + target_col):
    print(f"{col}:\n{full_data[col].value_counts()}\n")

LicenseNo:
0        456
2505     198
3292     176
4640      83
10634     59
        ... 
35454      1
4025       1
8123       1
29409      1
18375      1
Name: LicenseNo, Length: 37395, dtype: int64

FacilityID:
22897    2871
7347     1463
16025     715
376       492
16058     433
         ... 
24669       1
14405       1
9499        1
26003       1
8673        1
Name: FacilityID, Length: 27277, dtype: int64

FacilityName:
21868    3502
16384    2409
6993     1455
15284     921
326       817
         ... 
6704        1
23385       1
2899        1
14900       1
9505        1
Name: FacilityName, Length: 26033, dtype: int64

Type:
RESTAURANT                      131182
GROCERY STORE                    25012
SCHOOL                           12149
CHILDREN'S SERVICES FACILITY      3122
BAKERY                            2882
                                 ...  
CAT/LIQUOR                           1
ART GALLERY                          1
HERBALIFE STORE                      1
CAR WASH     

## Feature Engineering

In [17]:
show_all(full_data.sample(5))

,ID,Date,LicenseNo,FacilityID,FacilityName,Type,Street,City,State,LocationID,Reason,SectionViolations,RiskLevel,Geo_Loc,Inspection_Results
45587,16612306740217,2018-01-24,32058,20299,17291,RESTAURANT,9648,id-11235901,id_1890134,81870.0,LICENSE,3.0,High,locid14746,1
29552,18462565643050,2018-02-14,29544,1416,1269,RESTAURANT,720,id-11235901,id_1890134,81888.0,CANVASS,3.0,High,locid334,1
62729,12665392544678,2010-06-07,1101,26366,25161,GROCERY STORE,12074,id-11235901,id_1890134,81886.0,CANVASS,18.0,Low,locid12390,1
13890,42234088417051,2014-06-24,18881,13210,12541,DAYCARE ABOVE AND UNDER 2 YEARS,3256,id-11235901,id_1890134,81859.0,CANVASS,33.0,High,locid10320,4
61570,11168420710625,2013-07-16,6185,25872,24710,RESTAURANT,6393,id-11235901,id_1890134,81904.0,COMPLAINT RE-INSPECTION,38.0,High,locid16136,4


In [18]:
full_data['SectionViolations_NA'] = np.where(pd.isnull(full_data['SectionViolations']), 1, 0)
full_data['Type_NA'] = np.where(pd.isnull(full_data['Type']), 1, 0)
full_data['LocationID_NA'] = np.where(pd.isnull(full_data['LocationID']), 1, 0)

### Fix Missing Data

In [19]:
# 81880.0 is missing in sequence order
full_data['LocationID'] = full_data['LocationID'].fillna(81880.0)

In [20]:
full_data['Type'] = full_data['Type'].fillna('Unknown')

In [21]:
# 46 is missing in sequence order
full_data['SectionViolations'] = full_data['SectionViolations'].fillna(0.0)

In [22]:
missing_stats = full_data.isnull().sum()
print(missing_stats[missing_stats != 0])

Series([], dtype: int64)


## Creating New Features

In [23]:
full_data = full_data.sort_values('Date')

In [24]:
full_data[['Reason_Category', 'Reason_Sub_Category']] = full_data['Reason'].str.split(" ",expand=True)[[0, 1]]
full_data['Reason_Sub_Category'] = full_data['Reason_Sub_Category'].fillna('Unknown')

In [25]:
full_data['Facility'] = full_data['LicenseNo'].astype('str') + '_' + full_data['FacilityID'].astype('str') + '_' + full_data['FacilityName'].astype('str')

In [26]:
cat_cols = cat_cols + ['Facility']

In [27]:
group_cols = ['LicenseNo', 'FacilityID', 'FacilityName', 'Reason_Category']

In [28]:
full_data['inspections_cumcount'] = full_data.groupby(group_cols)['ID'].cumcount()+1

In [29]:
full_data['total_inspections'] = full_data.groupby(group_cols)['ID'].transform('count')

In [30]:
full_data['inspections_remaining'] = full_data['total_inspections'] - full_data['inspections_cumcount']

In [31]:
full_data['Days_Since_inception'] = (full_data['Date'] - full_data['Date'].min()).dt.days
full_data['Days_remaining'] = (full_data['Date'].max() - full_data['Date']).dt.days
full_data['Ordinal_Date'] = full_data["Date"].apply(lambda x: x.toordinal())

In [32]:
full_data['Days_Since_Prev_Visit_Facility'] = (full_data['Date'] - full_data.groupby(group_cols)['Date'].shift(1)).dt.days
full_data['Days_To_Next_Visit_Facility'] = full_data.groupby(group_cols)['Days_Since_Prev_Visit_Facility'].shift(-1)
full_data['Days_Since_First_Visit_Facility'] = (full_data['Date'] - full_data.groupby(group_cols)['Date'].transform('min')).dt.days

In [33]:
full_data['Daily_Inspections'] = full_data.groupby(['Date'])['ID'].transform('count')

In [34]:
for col in cat_cols:
    full_data[f'{col}_Daily_Inspections'] = full_data.groupby(['Date', col])['ID'].transform('count')

In [35]:
full_data['Prev_Reason'] = full_data.groupby('Facility')['Reason'].shift(1)
full_data['Next_Reason'] = full_data.groupby('Facility')['Reason'].shift(-1)

In [36]:
for col in ['LocationID', 'Geo_Loc']:
    full_data[f'Prev_{col}'] = full_data.groupby(['LicenseNo'])[col].shift(1)
    full_data[f'Next_{col}'] = full_data.groupby(['LicenseNo'])[col].shift(-1)

In [37]:
for col in ['RiskLevel', 'SectionViolations']:
    full_data[f'Prev_{col}'] = full_data.groupby(group_cols)[col].shift(1)
    full_data[f'Next_{col}'] = full_data.groupby(group_cols)[col].shift(-1)

In [38]:
for col in ['Date', 'LicenseNo', 'FacilityID', 'FacilityName', 'Type', 'Street', 'LocationID', 'Geo_Loc', 'Facility']:
    
    full_data[f'{col}_unq_reasons'] = full_data.groupby(col)['Reason'].transform('nunique')
    full_data[f'{col}_reasons_cumcount'] = full_data.groupby([col, 'Reason'])['ID'].cumcount()+1
    
    full_data[f'{col}_unq_violations'] = full_data.groupby(col)['SectionViolations'].transform('nunique')
    full_data[f'{col}_min_violations'] = full_data.groupby(col)['SectionViolations'].transform('min')
    full_data[f'{col}_max_violations'] = full_data.groupby(col)['SectionViolations'].transform('max')
    full_data[f'{col}_avg_violations'] = full_data.groupby(col)['SectionViolations'].transform('mean')
    full_data[f'{col}_violations_cumcount'] = full_data.groupby([col, 'SectionViolations'])['ID'].cumcount()+1
    
    full_data[f'{col}_unq_risks'] = full_data.groupby(col)['RiskLevel'].transform('nunique')
    full_data[f'{col}_risks_cumcount'] = full_data.groupby([col, 'RiskLevel'])['ID'].cumcount()+1    

In [39]:
full_data = full_data.sort_index()

In [40]:
show_all(full_data[full_data['FacilityID'] == 18501].sort_values('Date'))

,ID,Date,LicenseNo,FacilityID,FacilityName,Type,Street,City,State,LocationID,Reason,SectionViolations,RiskLevel,Geo_Loc,Inspection_Results,SectionViolations_NA,Type_NA,LocationID_NA,Reason_Category,Reason_Sub_Category,Facility,inspections_cumcount,total_inspections,inspections_remaining,Days_Since_inception,Days_remaining,Ordinal_Date,Days_Since_Prev_Visit_Facility,Days_To_Next_Visit_Facility,Days_Since_First_Visit_Facility,Daily_Inspections,LicenseNo_Daily_Inspections,FacilityID_Daily_Inspections,FacilityName_Daily_Inspections,Type_Daily_Inspections,Street_Daily_Inspections,City_Daily_Inspections,State_Daily_Inspections,LocationID_Daily_Inspections,Reason_Daily_Inspections,RiskLevel_Daily_Inspections,Geo_Loc_Daily_Inspections,Facility_Daily_Inspections,Prev_Reason,Next_Reason,Prev_LocationID,Next_LocationID,Prev_Geo_Loc,Next_Geo_Loc,Prev_RiskLevel,Next_RiskLevel,Prev_SectionViolations,Next_SectionViolations,Date_unq_reasons,Date_reasons_cumcount,Date_unq_violations,Date_min_violations,Date_max_violations,Date_avg_violations,Date_violations_cumcount,Date_unq_risks,Date_risks_cumcount,LicenseNo_unq_reasons,LicenseNo_reasons_cumcount,LicenseNo_unq_violations,LicenseNo_min_violations,LicenseNo_max_violations,LicenseNo_avg_violations,LicenseNo_violations_cumcount,LicenseNo_unq_risks,LicenseNo_risks_cumcount,FacilityID_unq_reasons,FacilityID_reasons_cumcount,FacilityID_unq_violations,FacilityID_min_violations,FacilityID_max_violations,FacilityID_avg_violations,FacilityID_violations_cumcount,FacilityID_unq_risks,FacilityID_risks_cumcount,FacilityName_unq_reasons,FacilityName_reasons_cumcount,FacilityName_unq_violations,FacilityName_min_violations,FacilityName_max_violations,FacilityName_avg_violations,FacilityName_violations_cumcount,FacilityName_unq_risks,FacilityName_risks_cumcount,Type_unq_reasons,Type_reasons_cumcount,Type_unq_violations,Type_min_violations,Type_max_violations,Type_avg_violations,Type_violations_cumcount,Type_unq_risks,Type_risks_cumcount,Street_unq_reasons,Street_reasons_cumcount,Street_unq_violations,Street_min_violations,Street_max_violations,Street_avg_violations,Street_violations_cumcount,Street_unq_risks,Street_risks_cumcount,LocationID_unq_reasons,LocationID_reasons_cumcount,LocationID_unq_violations,LocationID_min_violations,LocationID_max_violations,LocationID_avg_violations,LocationID_violations_cumcount,LocationID_unq_risks,LocationID_risks_cumcount,Geo_Loc_unq_reasons,Geo_Loc_reasons_cumcount,Geo_Loc_unq_violations,Geo_Loc_min_violations,Geo_Loc_max_violations,Geo_Loc_avg_violations,Geo_Loc_violations_cumcount,Geo_Loc_unq_risks,Geo_Loc_risks_cumcount,Facility_unq_reasons,Facility_reasons_cumcount,Facility_unq_violations,Facility_min_violations,Facility_max_violations,Facility_avg_violations,Facility_violations_cumcount,Facility_unq_risks,Facility_risks_cumcount
180888,33310774079444,2010-12-15,6419,18501,17675,BAKERY,7781,id-11235901,id_1890134,81874.0,CANVASS,33.0,Medium,locid2788,-1,0,0,0,CANVASS,Unknown,6419_18501_17675,1,7,6,710,2916,734121,NaN,210.0,0,79,1,1,1,3,1,79,79,4,37,21,1,1,NaN,CANVASS,NaN,81874.0,NaN,locid2788,NaN,Medium,NaN,38.0,9,4,14.0,0.0,35.0,16.886076,1,3,3,2,1,6.0,3.0,38.0,28.0,1,1,1,2,1,6.0,3.0,38.0,28.0,1,1,1,2,1,6.0,3.0,38.0,28.0,1,1,1,15,233,49.0,0.0,60.0,18.362248,64,3,328,2,1,6.0,3.0,38.0,28.0,1,1,1,14,347,53.0,0.0,58.0,18.763845,110,3,142,2,1,6.0,3.0,38.0,28.0,1,1,1,2,1,6.0,3.0,38.0,28.0,1,1,1
136941,34280183587076,2011-07-13,6419,18501,17675,BAKERY,7781,id-11235901,id_1890134,81874.0,CANVASS,38.0,Medium,locid2788,4,0,0,0,CANVASS,Unknown,6419_18501_17675,2,7,5,920,2706,734331,210.0,886.0,210,65,1,1,1,3,1,64,65,8,31,18,1,1,CANVASS,CANVASS,81874.0,81874.0,locid2788,locid2788,Medium,Medium,33.0,18.0,9,8,18.0,0.0,38.0,17.815385,1,3,4,2,2,6.0,3.0,38.0,28.0,1,1,2,2,2,6.0,3.0,38.0,28.0,1,1,2,2,2,6.0,3.0,38.0,28.0,1,1,2,15,304,49.0,0.0,60.0,18.362248,9,3,401,2,2,6.0,3.0,38.0,28.0,1,1,2,14,493,53.0,0.0,58.0,18.763845,32,3,176,2,2,6.0,3.0,38.0,28.0,1,1,2,2,2,6.0,3.0,38.0,28.0,1,1,2


In [41]:
generate_date_features(full_data, 'Date')

## Modeling

In [42]:
import sys
sys.path.append('ml_modules/')

In [43]:
from custom_estimator import Estimator
from encoding import FreqeuncyEncoding
from custom_fold_generator import FoldScheme
from lightgbm import LGBMClassifier
import category_encoders as enc
from sklearn.metrics import log_loss
from sklearn import model_selection

Using TensorFlow backend.


In [44]:
train_data = full_data[full_data['Inspection_Results'] != -1]
test_data = full_data[full_data['Inspection_Results'] == -1]

In [45]:
new_cat_cols = ['Reason_Category', 'Reason_Sub_Category', 'Prev_Reason', 'Next_Reason'
                , 'Prev_Geo_Loc', 'Next_Geo_Loc', 'Prev_RiskLevel', 'Next_RiskLevel'
               ]
categorical_cols = cat_cols + new_cat_cols

In [46]:
fE = FreqeuncyEncoding(categorical_columns=categorical_cols, return_df=True)
train_data = fE.fit_transform(train_data)
test_data = fE.transform(test_data)

In [47]:
drop_cols = id_col + target_col
y = train_data['Inspection_Results']

train_data.drop(columns=drop_cols, axis=1, inplace=True)
test_data.drop(columns=drop_cols, axis=1, inplace=True)

In [48]:
for df in (train_data, test_data):
    missing_stats = df.isnull().mean()
    print(missing_stats[missing_stats != 0])

Days_Since_Prev_Visit_Facility    0.341108
Days_To_Next_Visit_Facility       0.340043
Prev_LocationID                   0.190311
Next_LocationID                   0.189911
Prev_SectionViolations            0.341108
Next_SectionViolations            0.340043
dtype: float64
Days_Since_Prev_Visit_Facility    0.338956
Days_To_Next_Visit_Facility       0.342150
Prev_LocationID                   0.189937
Next_LocationID                   0.191137
Prev_SectionViolations            0.338956
Next_SectionViolations            0.342150
dtype: float64


In [49]:
train_data.shape, test_data.shape

((147443, 144), (49148, 144))

In [50]:
show_all(train_data.head())

,LicenseNo,FacilityID,FacilityName,Type,Street,City,State,LocationID,Reason,SectionViolations,RiskLevel,Geo_Loc,SectionViolations_NA,Type_NA,LocationID_NA,Reason_Category,Reason_Sub_Category,Facility,inspections_cumcount,total_inspections,inspections_remaining,Days_Since_inception,Days_remaining,Ordinal_Date,Days_Since_Prev_Visit_Facility,Days_To_Next_Visit_Facility,Days_Since_First_Visit_Facility,Daily_Inspections,LicenseNo_Daily_Inspections,FacilityID_Daily_Inspections,FacilityName_Daily_Inspections,Type_Daily_Inspections,Street_Daily_Inspections,City_Daily_Inspections,State_Daily_Inspections,LocationID_Daily_Inspections,Reason_Daily_Inspections,RiskLevel_Daily_Inspections,Geo_Loc_Daily_Inspections,Facility_Daily_Inspections,Prev_Reason,Next_Reason,Prev_LocationID,Next_LocationID,Prev_Geo_Loc,Next_Geo_Loc,Prev_RiskLevel,Next_RiskLevel,Prev_SectionViolations,Next_SectionViolations,Date_unq_reasons,Date_reasons_cumcount,Date_unq_violations,Date_min_violations,Date_max_violations,Date_avg_violations,Date_violations_cumcount,Date_unq_risks,Date_risks_cumcount,LicenseNo_unq_reasons,LicenseNo_reasons_cumcount,LicenseNo_unq_violations,LicenseNo_min_violations,LicenseNo_max_violations,LicenseNo_avg_violations,LicenseNo_violations_cumcount,LicenseNo_unq_risks,LicenseNo_risks_cumcount,FacilityID_unq_reasons,FacilityID_reasons_cumcount,FacilityID_unq_violations,FacilityID_min_violations,FacilityID_max_violations,FacilityID_avg_violations,FacilityID_violations_cumcount,FacilityID_unq_risks,FacilityID_risks_cumcount,FacilityName_unq_reasons,FacilityName_reasons_cumcount,FacilityName_unq_violations,FacilityName_min_violations,FacilityName_max_violations,FacilityName_avg_violations,FacilityName_violations_cumcount,FacilityName_unq_risks,FacilityName_risks_cumcount,Type_unq_reasons,Type_reasons_cumcount,Type_unq_violations,Type_min_violations,Type_max_violations,Type_avg_violations,Type_violations_cumcount,Type_unq_risks,Type_risks_cumcount,Street_unq_reasons,Street_reasons_cumcount,Street_unq_violations,Street_min_violations,Street_max_violations,Street_avg_violations,Street_violations_cumcount,Street_unq_risks,Street_risks_cumcount,LocationID_unq_reasons,LocationID_reasons_cumcount,LocationID_unq_violations,LocationID_min_violations,LocationID_max_violations,LocationID_avg_violations,LocationID_violations_cumcount,LocationID_unq_risks,LocationID_risks_cumcount,Geo_Loc_unq_reasons,Geo_Loc_reasons_cumcount,Geo_Loc_unq_violations,Geo_Loc_min_violations,Geo_Loc_max_violations,Geo_Loc_avg_violations,Geo_Loc_violations_cumcount,Geo_Loc_unq_risks,Geo_Loc_risks_cumcount,Facility_unq_reasons,Facility_reasons_cumcount,Facility_unq_violations,Facility_min_violations,Facility_max_violations,Facility_avg_violations,Facility_violations_cumcount,Facility_unq_risks,Facility_risks_cumcount,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Is_weekend
0,0.000081,0.000081,0.000081,0.667010,0.000081,0.998325,0.999722,0.007569,0.528631,33.0,0.717470,0.000081,0,0,0,0.635147,0.758144,0.000081,2,15,13,477,3149,733888,326.0,685.0,326,66,1,1,1,32,1,66,66,2,40,54,1,1,0.506002,0.566641,81876.0,81876.0,0.000092,0.000092,0.775376,0.775194,32.0,0.0,8,30,16.0,0.0,37.0,18.378788,6,3,43,2,2,10.0,0.0,43.0,22.666667,1,1,2,2,2,10.0,0.0,43.0,22.666667,1,1,2,2,2,10.0,0.0,43.0,22.666667,1,1,2,17,6376,62.0,0.0,70.0,17.950229,1678,4,10758,2,2,10.0,0.0,43.0,22.666667,1,1,2,13,101,48.0,0.0,60.0,17.847723,19,3,145,2,2,10.0,0.0,43.0,22.666667,1,1,2,2,2,10.0,0.0,43.0,22.666667,1,1,2,2010,4,17,26,0,116,False,False,False,False,False,False,0
1,0.000129,0.000129,0.000129,0.127582,0.000129,0.998325,0.999722,0.018794,0.092049,33.0,0.717470,0.000129,0,0,0,0.130403,0.758144,0.000129,1,16,15,168,3458,733579,NaN,151.0,0,80,1,1,1,7,1,80,80,3,4,56,1,1,0.000000,0.101740,NaN,81862.0,0.000000,0.000151,0.000000,0.775194,NaN,34.0,11,2,22.0,0.0,41.0,19.350000,4,3,16,5,1,11.0,0.0,55.0,26.000000,1,1,1,5,1,11.0,0.0,55.0,26.000000,1

In [51]:
lgb_params = {
    'n_estimators': 20000, 
    'learning_rate': 0.01,
    'boosting_type': 'gbdt', 
    'colsample_bytree': 0.80, 
    'min_child_weight': 9.0, 
    'num_leaves': 64, 
    'objective': 'multiclass', 
    'num_class': 7,
    'subsample': 0.80, 
    'subsample_freq': 5,
#     'n_jobs': -1
    }

lgb_100 = Estimator(LGBMClassifier(**lgb_params)
                     , random_state=100
                     , eval_metric='multi_logloss'
                     , scoring_metric=log_loss
                     , early_stopping_rounds=200
                     , verbose=400
                    )

In [52]:
lgb_oof_100 = lgb_100.fit_transform(train_data, y.values)

Training until validation scores don't improve for 200 rounds
[400]	train's multi_logloss: 0.129444	valid's multi_logloss: 0.156687
[800]	train's multi_logloss: 0.0787235	valid's multi_logloss: 0.13026
[1200]	train's multi_logloss: 0.0558646	valid's multi_logloss: 0.127254
[1600]	train's multi_logloss: 0.040985	valid's multi_logloss: 0.126769
Early stopping, best iteration is:
[1590]	train's multi_logloss: 0.0412891	valid's multi_logloss: 0.126753
Training until validation scores don't improve for 200 rounds
[400]	train's multi_logloss: 0.129196	valid's multi_logloss: 0.157292
[800]	train's multi_logloss: 0.0788305	valid's multi_logloss: 0.132098
[1200]	train's multi_logloss: 0.0560575	valid's multi_logloss: 0.129441
[1600]	train's multi_logloss: 0.041187	valid's multi_logloss: 0.129013
Early stopping, best iteration is:
[1633]	train's multi_logloss: 0.0402194	valid's multi_logloss: 0.128977
Training until validation scores don't improve for 200 rounds
[400]	train's multi_logloss: 0.12

In [53]:
lgb_200 = Estimator(LGBMClassifier(**lgb_params)
                     , random_state=200
                     , eval_metric='multi_logloss'
                     , scoring_metric=log_loss
                     , early_stopping_rounds=200
                     , verbose=400
                    )

In [54]:
lgb_oof_200 = lgb_200.fit_transform(train_data, y.values)

Training until validation scores don't improve for 200 rounds
[400]	train's multi_logloss: 0.129554	valid's multi_logloss: 0.155847
[800]	train's multi_logloss: 0.0790026	valid's multi_logloss: 0.129769
[1200]	train's multi_logloss: 0.0561278	valid's multi_logloss: 0.127063
[1600]	train's multi_logloss: 0.041278	valid's multi_logloss: 0.126809
Early stopping, best iteration is:
[1532]	train's multi_logloss: 0.0433961	valid's multi_logloss: 0.126744
Training until validation scores don't improve for 200 rounds
[400]	train's multi_logloss: 0.129633	valid's multi_logloss: 0.154985
[800]	train's multi_logloss: 0.078925	valid's multi_logloss: 0.130314
[1200]	train's multi_logloss: 0.0561802	valid's multi_logloss: 0.12808
Early stopping, best iteration is:
[1389]	train's multi_logloss: 0.0484235	valid's multi_logloss: 0.127935
Training until validation scores don't improve for 200 rounds
[400]	train's multi_logloss: 0.130227	valid's multi_logloss: 0.153175
[800]	train's multi_logloss: 0.0797

In [55]:
lgb_100.avg_cv_score, lgb_200.avg_cv_score

(0.12612189549576774, 0.1267232001052304)

In [56]:
lgb_100.feature_importances(columns=train_data.columns).head(10)

,column,feature_importance,rank
25,Days_To_Next_Visit_Facility,0.043370,1
9,SectionViolations,0.039972,2
21,Days_Since_inception,0.026568,3
110,LocationID_violations_cumcount,0.026075,4
24,Days_Since_Prev_Visit_Facility,0.026025,5
41,Next_Reason,0.022580,6
92,Type_violations_cumcount,0.022423,7
55,Date_avg_violations,0.021409,8
105,LocationID_reasons_cumcount,0.020989,9
112,LocationID_risks_cumcount,0.019158,10


In [57]:
lgb_pred_100 = lgb_100.transform(test_data)
lgb_pred_200 = lgb_200.transform(test_data)

In [58]:
final_sub = pd.DataFrame(data=np.mean([lgb_pred_100, lgb_pred_200], axis=0)
                         , columns=submission.columns)

In [59]:
final_sub.head()

,FACILITY CHANGED,FAIL,FURTHER INSPECTION REQUIRED,INSPECTION OVERRULED,PASS,PASS(CONDITIONAL),SHUT-DOWN
0,0.000062,0.965874,0.000233,0.002291,0.001675,0.029538,0.000327
1,0.000006,0.000326,0.000005,0.000016,0.999351,0.000282,0.000014
2,0.000080,0.000764,0.000331,0.980636,0.010851,0.000648,0.006690
3,0.002424,0.047872,0.000059,0.000134,0.040074,0.001468,0.907970
4,0.000002,0.000161,0.000005,0.000014,0.999232,0.000571,0.000015


In [60]:
file_timestamp = datetime.now().strftime('%m%d%Y_%H%M')
model_name='LGBM'

In [61]:
final_sub.to_excel(f"output/{model_name}_{file_timestamp}.xlsx", index=False)